# Transfer Learning With YAMNet for Heart Sound Classification

[YAMNet](https://tfhub.dev/google/yamnet/1) is a pre-trained deep neural network that can predict audio events from [521 classes](https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/yamnet_class_map.csv), such as laughter, barking, or a siren.

 In this tutorial you will learn how to:

- Load and use the YAMNet model for inference.
- Build a new model using the YAMNet embeddings to classify heart sounds.
- Evaluate and export your model.


## Import TensorFlow and other libraries


Start by installing [TensorFlow I/O](https://www.tensorflow.org/io), which will make it easier for you to load audio files off disk.

In [1]:
# Consolidate installations to avoid version conflicts.
# Attempting a clean reinstall of core libraries to resolve numpy compatibility issues.
# This step is crucial and must be run before any imports.
!pip uninstall -y numpy pandas tensorflow tensorflow-io tf-keras
!pip install -q tensorflow==2.17.1
!pip install -q tensorflow-io==0.37.1 tf-keras==2.17.0
!pip install -q pandas

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.3.3
Uninstalling pandas-2.3.3:
  Successfully uninstalled pandas-2.3.3
Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
Found existing installation: tensorflow-io 0.37.1
Uninstalling tensorflow-io-0.37.1:
  Successfully uninstalled tensorflow-io-0.37.1
Found existing installation: tf_keras 2.17.0
Uninstalling tf_keras-2.17.0:
  Successfully uninstalled tf_keras-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.11.0 requires pandas>=2.2, which is not installed.
cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, which is not installed.
geemap 0.35.3 requires pandas, which is not installed.
plotnine 0.14.5 requires pandas>=2.2.0,

In [2]:
# This cell is now redundant and causing conflicts, so its content is removed.

In [3]:
# This cell is now redundant and causing conflicts, so its content is removed.

In [4]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

ValueError: JAX requires ml_dtypes version 0.5 or newer; installed version is 0.4.1.

## About YAMNet

[YAMNet](https://github.com/tensorflow/models/tree/master/research/audioset/yamnet) is a pre-trained neural network that employs the [MobileNetV1](https://arxiv.org/abs/1704.04861) depthwise-separable convolution architecture. It can use an audio waveform as input and make independent predictions for each of the 521 audio events from the [AudioSet](http://g.co/audioset) corpus.

Internally, the model extracts "frames" from the audio signal and processes batches of these frames. This version of the model uses frames that are 0.96 second long and extracts one frame every 0.48 seconds .

The model accepts a 1-D float32 Tensor or NumPy array containing a waveform of arbitrary length, represented as single-channel (mono) 16 kHz samples in the range `[-1.0, +1.0]`. This tutorial contains code to help you convert WAV files into the supported format.

The model returns 3 outputs, including the class scores, embeddings (which you will use for transfer learning), and the log mel [spectrogram](https://www.tensorflow.org/tutorials/audio/simple_audio#spectrogram). You can find more details [here](https://tfhub.dev/google/yamnet/1).

One specific use of YAMNet is as a high-level feature extractor - the 1,024-dimensional embedding output. You will use the base (YAMNet) model's input features and feed them into your shallower model consisting of one hidden `tf.keras.layers.Dense` layer. Then, you will train the network on a small amount of data for audio classification _without_ requiring a lot of labeled data and training end-to-end. (This is similar to [transfer learning for image classification with TensorFlow Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub) for more information.)

First, you will test the model and see the results of classifying audio. You will then construct the data pre-processing pipeline.

### Loading YAMNet from TensorFlow Hub

You are going to use a pre-trained YAMNet from [Tensorflow Hub](https://tfhub.dev/) to extract the embeddings from the sound files.

Loading a model from TensorFlow Hub is straightforward: choose the model, copy its URL, and use the `load` function.

Note: to read the documentation of the model, use the model URL in your browser.

In [ ]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
_ = tf.keras.utils.get_file('esc-50.zip',
                        'https://github.com/ShebinJoseJacob/test/raw/main/Data.zip',
                        cache_dir='./',
                        cache_subdir='datasets',
                        extract=True)

With the model loaded, you can follow the [YAMNet basic usage tutorial](https://www.tensorflow.org/hub/tutorials/yamnet) and download a sample WAV file to run the inference.


In [ ]:
testing_wav_file_name = tf.keras.utils.get_file('miaow_16k.wav',
                                                'https://storage.googleapis.com/audioset/miaow_16k.wav',
                                                cache_dir='./',
                                                cache_subdir='test_data')

print(testing_wav_file_name)

You will need a function to load audio files, which will also be used later when working with the training data. (Learn more about reading audio files and their labels in [Simple audio recognition](https://www.tensorflow.org/tutorials/audio/simple_audio#reading_audio_files_and_their_labels).

Note: The returned `wav_data` from `load_wav_16k_mono` is already normalized to values in the `[-1.0, 1.0]` range (for more information, go to [YAMNet's documentation on TF Hub](https://tfhub.dev/google/yamnet/1)).

In [ ]:
# Utility functions for loading audio files and making sure the sample rate is correct.

@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
testing_wav_data = load_wav_16k_mono('/content/datasets/esc-50_extracted/Data/audio/New_MS_074.wav')

_ = plt.plot(testing_wav_data)

# Play the audio file.
display.Audio(testing_wav_data, rate=16000)

### Load the class mapping

It's important to load the class names that YAMNet is able to recognize. The mapping file is present at `yamnet_model.class_map_path()` in the CSV format.

In [ ]:
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

for name in class_names[:20]:
  print(name)
print('...')

### Run inference

YAMNet provides frame-level class-scores (i.e., 521 scores for every frame). In order to determine clip-level predictions, the scores can be aggregated per-class across frames (e.g., using mean or max aggregation). This is done below by `scores_np.mean(axis=0)`. Finally, to find the top-scored class at the clip-level, you take the maximum of the 521 aggregated scores.


In [ ]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.math.argmax(class_scores)
inferred_class = class_names[top_class]

print(f'The main sound is: {inferred_class}')
print(f'The embeddings shape: {embeddings.shape}')

Note: The model correctly inferred an animal sound. Your goal in this tutorial is to increase the model's accuracy for specific classes. Also, notice that the model generated 13 embeddings, 1 per frame.

## Heart Sound dataset

The [dataset](https://github.com/Nekhil-haxh/Sign-Language-Translator) is a labeled collection of 1,000 five-second long environmental audio recordings. The dataset consists of 5 classes, with 200 examples per class.

Download the dataset and extract it.


In [ ]:
'''_ = tf.keras.utils.get_file('esc-50.zip',
                        'https://github.com/ShebinJoseJacob/test/raw/main/Data.zip',
                        cache_dir='./',
                        cache_subdir='datasets',
                        extract=True)'''

### Explore the data

The metadata for each file is specified in the csv file at `./datasets/Data/meta/esc50.csv`

and all the audio files are in `./datasets/Data/audio/`

You will create a pandas `DataFrame` with the mapping and use that to have a clearer view of the data.


In [ ]:
esc50_csv = '/content/datasets/esc-50_extracted/Data/meta/metadata.csv'
base_data_path = '/content/datasets/esc-50_extracted/Data/audio'

pd_data = pd.read_csv(esc50_csv)
pd_data.head()

### Filter the data

Now that the data is stored in the `DataFrame`, apply some transformations:

- Filter out rows and use only the selected classes
- Amend the filename to have the full path. This will make loading easier later.
- Change targets to be within a specific range.

In [ ]:
my_classes = ['AS', 'MR', 'MS', 'MVP', 'N']
map_class_to_id = {'AS':0, 'MR':1, 'MS':2, 'MVP':3, 'N': 4}

filtered_pd = pd_data[pd_data.foldername.isin(my_classes)]

class_id = filtered_pd['foldername'].apply(lambda name: map_class_to_id[name])
filtered_pd = filtered_pd.assign(target=class_id)

full_path = filtered_pd['file name'].apply(lambda row: os.path.join(base_data_path, row))
filtered_pd = filtered_pd.assign(filename=full_path)

filtered_pd.head(1000)

### Load the audio files and retrieve embeddings

Here you'll apply the `load_wav_16k_mono` and prepare the WAV data for the model.

When extracting embeddings from the WAV data, you get an array of shape `(N, 1024)`  where `N` is the number of frames that YAMNet found (one for every 0.48 seconds of audio).

Your model will use each frame as one input. Therefore, you need to create a new column that has one frame per row. You also need to expand the labels and the `fold` column to proper reflect these new rows.

The expanded `fold` column keeps the original values. You cannot mix frames because, when performing the splits, you might end up having parts of the same audio on different splits, which would make your validation and test steps less effective.

In [ ]:
filenames = filtered_pd['filename']
targets = filtered_pd['target']
folds = filtered_pd['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec
print(main_ds.take(1000))

In [ ]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

In [ ]:
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

### Split the data

You will use the `fold` column to split the dataset into train, validation and test sets.

The last step is to remove the `fold` column from the dataset since you're not going to use it during training.


In [ ]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 4)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 4)
test_ds = cached_ds.filter(lambda embedding, label, fold: fold == 5)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(100).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

## Create your model

You did most of the work!
Next, define a very simple [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) model with one hidden layer and two outputs to recognize cats and dogs from sounds.


In [ ]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024,), dtype=tf.float32, name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()


In [ ]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [ ]:
history = my_model.fit(train_ds,
                       epochs=200,
                       validation_data=val_ds,
                       callbacks=callback)

Let's run the `evaluate` method on the test data just to be sure there's no overfitting.

In [ ]:
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

You did it!

In [ ]:
saved_model_path = "./saved_model"

# Save the model
my_model.save("saved_model.keras")


## Test your model

Next, try your model on the embedding from the previous test using YAMNet only.


In [ ]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
result = my_model(embeddings).numpy()

print(result)

inferred_class = my_classes[result.mean(axis=0).argmax()]
print(f'The main sound is: {inferred_class}')

## Save a model that can directly take a WAV file as input

Your model works when you give it the embeddings as input.

In a real-world scenario, you'll want to use audio data as a direct input.

To do that, you will combine YAMNet with your model into a single model that you can export for other applications.

To make it easier to use the model's result, the final layer will be a `reduce_mean` operation. When using this model for serving (which you will learn about later in the tutorial), you will need the name of the final layer. If you don't define one, TensorFlow will auto-define an incremental one that makes it hard to test, as it will keep changing every time you train the model. When using a raw TensorFlow operation, you can't assign a name to it. To address this issue, you'll create a custom layer that applies `reduce_mean` and call it `'classifier'`.


In [ ]:
@tf.keras.saving.register_keras_serializable()
class ReduceMeanLayer(tf.keras.layers.Layer):
  def __init__(self, axis=0, **kwargs):
    super(ReduceMeanLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.math.reduce_mean(input, axis=self.axis)

  def get_config(self):
    config = super().get_config()
    config.update({
        "axis": self.axis,
    })
    return config

In [ ]:
saved_model_path = './dogs_and_cats_yamnet_test.keras'

@tf.keras.saving.register_keras_serializable()
class YAMNetWrapperLayer(tf.keras.layers.Layer):
  def __init__(self, yamnet_model_handle, **kwargs):
    super(YAMNetWrapperLayer, self).__init__(**kwargs)
    self.yamnet_model_handle = yamnet_model_handle
    self.yamnet_model = hub.load(yamnet_model_handle)

  def call(self, inputs):
    # yamnet_model expects a float32 tensor
    scores, embeddings, spectrogram = self.yamnet_model(inputs)
    return scores, embeddings, spectrogram

  def get_config(self):
    config = super().get_config()
    config.update({
        "yamnet_model_handle": self.yamnet_model_handle,
    })
    return config

  @classmethod
  def from_config(cls, config):
    yamnet_model_handle = config.pop("yamnet_model_handle")
    return cls(yamnet_model_handle, **config)

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')

# Use the custom wrapper layer with the yamnet_model_handle
yamnet_wrapper_layer = YAMNetWrapperLayer(yamnet_model_handle, name='yamnet')
_, embeddings_output, _ = yamnet_wrapper_layer(input_segment)

serving_outputs = my_model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)
serving_model.save('final_model.h5', include_optimizer=False)

Load your saved model to verify that it works as expected.

In [ ]:
@tf.keras.saving.register_keras_serializable()
class SplitLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.split(inputs, num_or_size_splits=5, axis=-1)

@tf.keras.saving.register_keras_serializable()
class SqueezeLayer(tf.keras.layers.Layer):
  def __init__(self, axis=None, **kwargs):
    super(SqueezeLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.squeeze(input, axis=self.axis)

  def get_config(self):
    config = super().get_config()
    config.update({
        "axis": self.axis,
    })
    return config

In [ ]:
split_model_path = './dogs_and_cats_yamnet_split.keras'

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
# Use the custom YAMNetWrapperLayer instead of hub.KerasLayer
yamnet_wrapper_layer_for_split = YAMNetWrapperLayer(yamnet_model_handle, name='yamnet_for_split')
_, embeddings_output, _ = yamnet_wrapper_layer_for_split(input_segment)
serving_outputs = my_model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_outputs = SplitLayer()(serving_outputs)
# Use the new SqueezeLayer for each output
serving_outputs = [SqueezeLayer(axis=-1)(output) for output in serving_outputs]
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(split_model_path, include_optimizer=False)
serving_model.save('final_model.h5', include_optimizer=False)

In [ ]:
reloaded_model = tf.keras.models.load_model(saved_model_path)

In [ ]:
split_model = tf.keras.models.load_model(split_model_path)

In [ ]:
testing_wav_data = load_wav_16k_mono(os.path.join(base_data_path, 'New_AS_171.wav'))

And for the final test: given some sound data, does your model return the correct result?

In [ ]:
reloaded_results = reloaded_model(testing_wav_data)

print(reloaded_results)
cat_or_dog = my_classes[tf.math.argmax(reloaded_results)]
print(f'The main sound is: {cat_or_dog}')

In [ ]:
split_results = split_model(testing_wav_data)

print(split_results)

If you want to try your new model on a serving setup, you can use the 'serving_default' signature.

In [ ]:
serving_results = reloaded_model(testing_wav_data)
cat_or_dog = my_classes[tf.math.argmax(serving_results)]
print(f'The main sound is: {cat_or_dog}')

## (Optional) Some more testing

The model is ready.

Let's compare it to YAMNet on the test dataset.

In [ ]:
test_pd = filtered_pd.loc[filtered_pd['fold'] == 5]
row = test_pd.sample(1)
filename = row['filename'].item()
print(filename)
waveform = load_wav_16k_mono(filename)
print(f'Waveform values: {waveform}')
_ = plt.plot(waveform)

display.Audio(waveform, rate=16000)

In [ ]:
# Run the model, check the output.
scores, embeddings, spectrogram = yamnet_model(waveform)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.math.argmax(class_scores)
inferred_class = class_names[top_class]
top_score = class_scores[top_class]
print(f'[YAMNet] The main sound is: {inferred_class} ({top_score})')

reloaded_results = reloaded_model(waveform)
your_top_class = tf.math.argmax(reloaded_results)
your_inferred_class = my_classes[your_top_class]
class_probabilities = tf.nn.softmax(reloaded_results, axis=-1)
your_top_score = class_probabilities[your_top_class]
print(f'[Your model] The main sound is: {your_inferred_class} ({your_top_score})')

In [ ]:
model.summary()


In [ ]:
# ============================================================
#   EXTRAER MÉTRICAS PARA DOCUMENTO DEL PROYECTO
# ============================================================

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# ----- 1. Obtener predicciones del modelo en test -----

y_true = []
y_pred = []

for embeddings_batch, labels_batch in test_ds:
    preds = my_model.predict(embeddings_batch)
    preds = np.argmax(preds, axis=1)

    y_true.extend(labels_batch.numpy())
    y_pred.extend(preds)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# ----- 2. Matriz de confusión -----

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=my_classes,
            yticklabels=my_classes)
plt.title("Matriz de Confusión del Modelo")
plt.xlabel("Predicción")
plt.ylabel("Verdadero")
plt.show()

# ----- 3. Precision, Recall, F1-Score -----

print("\n=== Reporte de Clasificación ===\n")
print(classification_report(y_true, y_pred, target_names=my_classes))

# ----- 4. Gráficas de entrenamiento -----

# Accuracy
plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Evolución de Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Evolución de la Pérdida (Loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# ----- 5. Tamaño del dataset y splits -----

print("\nTamaño de cada partición:\n")
print("Entrenamiento:", len(list(train_ds.unbatch())))
print("Validación:", len(list(val_ds.unbatch())))
print("Prueba:", len(list(test_ds.unbatch())))
